## Segmenting and Clustering Neighborhoods in Toronto - Part 1

#### Import libraries

In [7]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [8]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

#### Download and scrape the data

In [10]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text

In [14]:
soup = BeautifulSoup(source)

In [15]:
table = soup.find('table')

In [16]:
c_names = ['Postal Code','Borough','Neighborhood']
df = pd.DataFrame(columns = c_names)

In [17]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [18]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Clean data

In [20]:
df = df.drop(df[(df.Borough == "Not assigned")].index)
df.Neighborhood.replace("Not assigned", df.Borough, inplace=True)

df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [24]:
df2=df.groupby('Postal Code')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
df2=df2.reset_index(drop=False)
df2.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [25]:
df3 = pd.merge(df, df2, on='Postal Code')
df3.drop(['Neighborhood'],axis=1,inplace=True)
df3.drop_duplicates(inplace=True)
df3.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)
df3.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [26]:
df3.shape

(103, 3)